## Overview

## Setup

In [1]:
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from PIL import Image
from matplotlib import cm

import numpy as np
from sklearn.metrics import accuracy_score

print(tf.__version__)

2.6.0


Define model

In [2]:
#model_path = '../frozen_base/frozen_graph.pb'
#model_path = './frozen_pruned/frozen_graph.pb'

In [3]:
def get_ds():
    """
    ## Generate a `Dataset`
    """

    image_size = (180, 180)
    batch_size = 32

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "../PetImages",
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "../PetImages",
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )

    def scale(image, label):
        return tf.image.convert_image_dtype(image, tf.float32)/255, label

    train_ds = train_ds.map(scale)
    val_ds = val_ds.map(scale)
    
    return train_ds, val_ds

In [4]:
def evaluate_model(model_reloaded):
    global output
    

    # Run predictions on ever y image in the "test" dataset.
    prediction = []
    y_true = []
    
    

    
    for i, test_image in enumerate(tqdm(test_images)):    
        #if i % 100 == 0:
        #    print('Evaluated on {n} results so far.'.format(n=i))
            
            
        if i == 500:  # Ограничемся 500-ми первыми (очень долго ждать всех...)          
            break
                    
        #r,g,b = cv2.split(test_image)
        #img = cv2.merge([b,g,r])
                

        # Use the given image as input, which needs to be blob(s).
        #tensorflowNet.setInput(cv2.dnn.blobFromImage(img, size=(180, 180), swapRB=True, crop=False))


        # Runs a forward pass to compute the net output
        #networkOutput = tensorflowNet.forward()

        #output = networkOutput[0]    
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)        
        output = model_reloaded.predict(test_image)           
        
        
        
        if output[0] > 0.5:
            prediction.append(1)
        else:
            prediction.append(0)            
        y_true.append(test_labels[i])

    print('\n')
    prediction = np.array(prediction)
    y_true = np.array(y_true)    
    accuracy = accuracy_score(y_true, prediction)
    return accuracy

In [5]:
_, val_ds = get_ds()


test_images = []
test_labels =[]
for element in val_ds.as_numpy_iterator():

    temp = list(element[1].reshape(-1))
    test_labels = test_labels + temp
    a = element[0]
    temp = list(a)  
    test_images = test_images + temp    


test_images = np.array(test_images)
test_labels = np.array(test_labels)    

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Found 23422 files belonging to 2 classes.
Using 4684 files for validation.


In [6]:
model_reloaded = tf.keras.models.load_model('../base')

In [7]:
%%time
with tf.device('/cpu:0'):
    test_accuracy = evaluate_model(model_reloaded)

 11%|████████████████████▋                                                                                                                                                                             | 500/4684 [00:30<04:12, 16.58it/s]



Wall time: 30.2 s


In [10]:
print('Base test accuracy:', test_accuracy)
print('File size:', 34768813, 'bytes.')

Base test accuracy: 0.952
File size: 34768813 bytes.
